In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import mlflow

In [ ]:
mlflow.set_tracking_uri(f"http://model-platform.com/registry/grosse-conf-25/")

In [ ]:
import pandas as pd
import mlflow.pyfunc
from transformers import pipeline

model = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

class HFModelWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        self.model = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

    def predict(self, model_input):
        if isinstance(model_input, dict):
            input_df = pd.DataFrame([model_input])
        else:
            input_df = pd.DataFrame(model_input)
        return self.model(input_df.iloc[:, 0].tolist())

mlflow.pyfunc.log_model(python_model=HFModelWrapper(), artifact_path="custom_model", registered_model_name="huggingface_model")
